## Imports

In [1]:
import os
import numpy as np
import pandas as pd
import hypertools as hyp
from umap import UMAP
from os.path import join as opj

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Set paths

In [2]:
datadir = '../data-analysis/'
traj_dir = opj(datadir, 'trajectories')
fig_dir = '../../figures/'

## Load trajectories

In [3]:
forces_vid = np.load(opj(traj_dir, 'forces_v_traj.npy'))
bos_vid = np.load(opj(traj_dir, 'bos_v_traj.npy'))
forces_qs = np.load(opj(traj_dir, 'forces_q_traj.npy'))
bos_qs = np.load(opj(traj_dir, 'bos_q_traj.npy'))

qsets = ['qset1', 'qset2', 'qset3']
forces_traces = dict.fromkeys(qsets)
bos_traces = dict.fromkeys(qsets)
for qset in range(1, 4):
    fq_path = opj(traj_dir, f'forces_traces_set{qset - 1}.p')
    bq_path = opj(traj_dir, f'bos_traces_set{qset - 1}.p')
    forces_traces[f'qset{qset}'] = np.load(fq_path, allow_pickle=True)
    bos_traces[f'qset{qset}'] = np.load(bq_path, allow_pickle=True)

## Create 2D embeddings

In [4]:
seed = 0
umap_params = {
    'n_components': 2,
    'n_neighbors': 15,
    'spread': 1.0,
    'min_dist': 0.1,
    'metric': 'correlation',
    'init': 'random',
    'random_state': seed,
}

In [5]:
# embedding all videos, questions, traces together
to_reduce = [forces_vid] + [bos_vid] + [forces_qs] + [bos_qs]
for vid_traces in [forces_traces, bos_traces]:
    for qset_traces in vid_traces.values():
        for sub in sorted(list(qset_traces.keys())):
            to_reduce.append(qset_traces[sub])

In [6]:
split = np.cumsum([len(traj) for traj in to_reduce])[:-1]
stacked_trajs = np.vstack(to_reduce)

np.random.seed(seed)
reducer = UMAP(**umap_params)
stacked_embeddings = reducer.fit_transform(stacked_trajs)
embeddings = np.vsplit(stacked_embeddings, split)

In [7]:
np.save(opj(datadir, 'embeddings.npy'), embeddings)
np.save(opj(datadir, 'UMAP_reducer.npy'), reducer)
# embeddings = np.load(opj(datadir, 'embeddings.npy'))
# reducer = np.load(opj(datadir, 'UMAP_reducer.npy'), allow_pickle=True).item()

In [8]:
forces_vid_emb, bos_vid_emb, forces_qs_emb, bos_qs_emb = embeddings[:4]
forces_traces_embs = {qset: {sub: None for sub in range(1, 41)} for qset in qsets}
bos_traces_embs = {qset: {sub: None for sub in range(1, 41)} for qset in qsets}

ix = 4
for trace_emb_dict in [forces_traces_embs, bos_traces_embs]:
    for qset in qsets:
        for sub in sorted(list(forces_traces[qset].keys())):
            
            trace_emb_dict[qset][sub] = embeddings[ix]
            ix += 1

## Create a 2D grid

In [9]:
scale = np.max([np.abs(i) for i in [forces_vid_emb.min(), 
                                    forces_vid_emb.max(), 
                                    bos_vid_emb.min(), 
                                    bos_vid_emb.max()]]) // 1 + 1

step = scale / 25
print(f'scale: {scale}, step: {step}')

X, Y = np.meshgrid(np.arange(-scale, scale, step), np.arange(-scale, scale, step))

scale: 31.0, step: 1.24


In [25]:
reducer.inverse_transform

'/Users/paxtonfitzpatrick/opt/anaconda3/envs/py37/lib/python3.7/site-packages/umap/__init__.py'